In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, BertModel, AdamW;

import torch;
import torch.nn as nn;
import torch.nn.functional as F;
import torch.optim as optim;
from torch.utils.data import Dataset, DataLoader;

In [ ]:
import os;
import os.path;

In [ ]:
import numpy as np;
import matplotlib.pyplot as plt;

import statistics;

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu');
print(device);

cpu


In [ ]:
"""
    Using bert model for downstream classification task.
"""

class BERTClassifier(nn.Module):
    def __init__(self, model_name, num_classes):
        super(BERTClassifier, self).__init__();
        self.bert = BertModel.from_pretrained(model_name);
        self.dropout = nn.Dropout(p=0.2);
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes);

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids=input_ids, attention_mask=attention_mask);
        output = self.dropout(output.pooler_output);
        output = self.fc(output);
        return output;

In [ ]:
roberta_model_name = 'bert-base-uncased';
num_classes = 3;
max_length = 512;
batch_size = 16;
epochs = 3;
learning_rate = 1e-5;

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(roberta_model_name);

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class Dataset(nn.Module):
    def __init__(self,path_x,path_y,max_length):

        self.max_length = max_length;

        if not os.path.exists(path_x):
            raise FileNotFoundError(path_x);
        if not os.path.exists(path_y):
            raise FileNotFoundError(path_y);

        with open(path_x,'r') as f:
            self.x = f.readlines();

        with open(path_y,'r') as f:
            self.y = f.readlines();

        if len(self.x) != len(self.y):
            raise ValueError("x and y must have same length");

    def __len__(self):
        return len(self.x);

    def __getitem__(self,idx):
        x = self.x[idx];
        y = self.y[idx];
        output = tokenizer(x, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt');
        return {
            'input_ids': output['input_ids'].flatten(),
            'attention_mask': output['attention_mask'].flatten(),
            'label' : torch.tensor(int(self.y[idx]))
        };

In [ ]:
train_dataset = Dataset('/content/drive/MyDrive/Data/DATA/trainset_cmb.txt',
                        '/content/drive/MyDrive/Data/DATA/y_train.txt',max_length);

val_dataset = Dataset('/content/drive/MyDrive/Data/DATA/valset_cmb.txt',
                      '/content/drive/MyDrive/Data/DATA/y_val.txt',max_length);

train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True);
val_loader = DataLoader(val_dataset,batch_size=batch_size,shuffle=True);

In [ ]:
model = BERTClassifier(roberta_model_name, num_classes);
model = model.to(device);
optimizer = AdamW(model.parameters(), lr=learning_rate);
criterion = nn.CrossEntropyLoss();

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
store_best_path = "/content/drive/MyDrive/PARAM/";

In [ ]:
epochs = 5;

train_loss = [];
val_loss = [];
train_acc = [];
val_acc = [];

for epoch in range(epochs):

    train_loss_epoch = [];
    val_loss_epoch = [];
    train_acc_epoch = [];
    val_acc_epoch = [];

    model.train();
    for batch in train_loader:
        optimizer.zero_grad();

        input_ids = batch['input_ids'].to(device);
        attention_mask = batch['attention_mask'].to(device);
        y = batch['label'].to(device);

        output = model(input_ids,attention_mask);
        loss = criterion(output,y);
        loss.backward();
        optimizer.step();

        train_loss_epoch.append(loss.item());

        accuracy = torch.argmax(output,dim=-1).view(-1) == y.view(-1);
        train_acc_epoch.append((torch.sum(accuracy) / len(accuracy)).item());


    model.eval();
    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device);
            attention_mask = batch['attention_mask'].to(device);
            y = batch['label'].to(device);

            output = model(input_ids,attention_mask);
            loss = criterion(output,y);

            val_loss_epoch.append(loss.item());

            accuracy = torch.argmax(output,dim=-1).view(-1) == y.view(-1);
            val_acc_epoch.append((torch.sum(accuracy) / len(accuracy)).item());


    print(f"Epoch[{epoch}][{epochs}] : Training Loss :{statistics.mean(train_loss_epoch)}, Validation Loss :{statistics.mean(val_loss_epoch)}, \
Training Accuracy :{statistics.mean(train_acc_epoch)}, Validation Accuracy :{statistics.mean(val_acc_epoch)}");

    if len(val_loss) == 0 or statistics.mean(val_loss_epoch) < min(val_loss):
        torch.save(model.state_dict(),store_best_path + 'bert_wth_rouge_model.pt');

    train_loss.append(statistics.mean(train_loss_epoch));
    val_loss.append(statistics.mean(val_loss_epoch));
    train_acc.append(statistics.mean(train_acc_epoch));
    val_acc.append(statistics.mean(val_acc_epoch));

Epoch[0][5] : Training Loss :1.0215022578835486, Validation Loss :0.9859029173851013, Training Accuracy :0.43875, Validation Accuracy :0.47625
Epoch[1][5] : Training Loss :0.9657432550191879, Validation Loss :0.9518803668022155, Training Accuracy :0.50625, Validation Accuracy :0.52375
Epoch[2][5] : Training Loss :0.8809739750623703, Validation Loss :0.9381988370418548, Training Accuracy :0.579375, Validation Accuracy :0.54125
Epoch[3][5] : Training Loss :0.7553196761012078, Validation Loss :0.949640953540802, Training Accuracy :0.671875, Validation Accuracy :0.555
Epoch[4][5] : Training Loss :0.578577750325203, Validation Loss :1.0238290947675706, Training Accuracy :0.7678125, Validation Accuracy :0.5425


In [ ]:
class Dataset(nn.Module):
    def __init__(self,path_x,max_length):

        self.max_length = max_length;

        if not os.path.exists(path_x):
            raise FileNotFoundError(path_x);

        with open(path_x,'r') as f:
            self.x = f.readlines();


    def __len__(self):
        return len(self.x);

    def __getitem__(self,idx):
        x = self.x[idx];
        output = tokenizer(x, max_length=self.max_length, padding='max_length', truncation=True, return_tensors='pt');
        return {
            'input_ids': output['input_ids'].flatten(),
            'attention_mask': output['attention_mask'].flatten(),
        };

In [ ]:
test_dataset = Dataset('/content/drive/MyDrive/Webis/DATA/testset_cmb.txt',
                        max_length);

test_dataloader = DataLoader(test_dataset,batch_size=8);

In [ ]:
model = ROBERTaClassifier(roberta_model_name, num_classes);
model = model.to(device);
model.load_state_dict(torch.load(store_best_path + 'bert_wth_rouge_model.pt',map_location=device));

Some weights of RobertaModel were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
test = []
for batch in test_dataloader:
  model.eval();

  input_ids = batch['input_ids'].to(device);
  attention_mask = batch['attention_mask'].to(device);

  output = model(input_ids,attention_mask);
  y_pred = torch.argmax(output,dim=-1);
  test.append(y_pred);

In [ ]:
tmp = [];
for batch in test:
    for line in batch:
        tmp.append(line.item());

In [ ]:
id_to_label = {
    0:"passage",
    1:"phrase",
    2:"multi"
};

In [ ]:
test_pred_label = [id_to_label[_] for _ in tmp];

In [ ]:
data = {
    "id":range(len(test_pred_label)),
    "spoilerType":test_pred_label
};

In [ ]:
import pandas as pd;

In [ ]:
df = pd.DataFrame(data);
print(df);

      id spoilerType
0      0     passage
1      1     passage
2      2      phrase
3      3      phrase
4      4     passage
..   ...         ...
395  395     passage
396  396      phrase
397  397       multi
398  398     passage
399  399      phrase

[400 rows x 2 columns]


In [ ]:
df.to_csv("/content/drive/MyDrive/Webis/DATA/roberta_submission.csv",index=False);